In [1]:
import DataLoader
from importlib import reload
reload(DataLoader)
import os
import random

folder = "/mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks(5)"

files = [entry.path for entry in os.scandir(folder) if entry.is_file()]

if len(files) > 10000:
    files = random.sample(files, 10000)

data_graphs = []
for f in files:
    try:
        d = DataLoader.load_file(f)
        DataLoader.aggregate_sequences(d)
        data_graphs.append(d)
    except Exception as e:
        print(f"Fehler beim Laden von {f}: {e}")

data = []
for G in data_graphs:
    try:
        dat = DataLoader.graph_to_dataset(G)
        data.append(dat)
    except Exception as e:
        print(f"Fehler beim Laden von {G}: {e}")


"""
for node, attrs in data_graphs[0].nodes(data=True):
    seq = attrs.get("sequences", None)
    if seq is not None:
        print(node, seq[:50])   # nur die ersten 50 Stellen
"""

data_sample = random.choice(data)
#example = load_file(file)


/home/kippnich/miniconda3/envs/pangenome-hgt-sim/lib/python3.12/site-packages/numpy/core/getlimits.py:542: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)
/mnt/c/Users/uhewm/OneDrive/PhD/Project No.2/pangenome-gene-transfer-simulation/DataLoader.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edges = torch.tensor(graph_properties[1], dtype=torch.long)  # [2, num_edges]
/mnt/c/Users/uhewm/OneDrive/PhD/Project No.2/pangenome-gene-transfer-simulation/DataLoader.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad

Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks(5)/simulation_163730.h5: cannot reshape array of size 0 into shape (0,newaxis)
Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks(5)/simulation_191463.h5: cannot reshape array of size 0 into shape (0,newaxis)
Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks(5)/simulation_27197.h5: cannot reshape array of size 0 into shape (0,newaxis)
Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks(5)/simulation_80205.h5: cannot reshape array of size 0 into shape (0,newaxis)
Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks(5)/simulation_158726.h5: cannot reshape array of size 0 into shape (0,newaxis)
Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/simulation_chunks(5)/simulation_195334.h5: cannot reshape array of size 0 into shape (0,newaxis)
Fehler beim Laden von /mnt/c/Users/uhewm/Desktop/ProjectHGT/

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing


class ParentChildFusionLayer(MessagePassing):
    """
    A MessagePassing layer designed for tree-like graphs where each parent
    has either exactly two children or none. For each parent node i, the
    features of the parent and its two children (if present) are concatenated.

    This layer does not use any attention or permutation-invariant
    aggregation: child messages are collected explicitly and concatenated
    in a fixed order. The user must ensure that each parent node has either
    (0 or 2) incoming edges, and that the edge_index ordering encodes a
    consistent left/right child order.

    Input dimensions:
        - Node feature dimension: in_dim
        - Output feature dimension: out_dim

    Output:
        - New node embeddings of dimension out_dim
    """

    def __init__(self, in_dim):
        # We do not use built-in aggregation ("add", "mean", ...) because
        # we aggregate manually. Set aggr=None.
            
        # Each node will produce: [parent_features, child1, child2]
        # If a node has no children, child features are zero-padded.
        super().__init__(node_dim=0, aggr=None)

        self.in_dim = in_dim

    def message(self, x_j):
        return x_j

    def aggregate(self, inputs, index, ptr=None, dim_size=None):
        """
        Collect exactly two child feature vectors per parent.

        inputs:  (num_edges, in_dim)
        index:   (num_edges,) target node for each edge

        Returns:
            A tensor of shape (num_nodes, 2 * in_dim) containing the two
            children features for each parent. Order is determined by
            edge ordering and should be consistent in the dataset.
        """

        # Determine number of nodes from dim_size (preferred), fall back to index
        if dim_size is not None:
            num_nodes = dim_size
        else:
            num_nodes = int(index.max().item()) + 1

        device = inputs.device

        # Preallocate storage
        children = torch.zeros(num_nodes, 2, self.in_dim, device=device)

        # Compute for each edge its "child slot" 0 or 1
        # Example: for index = [3,3,5,5], this produces [0,1,0,1]
        slot = torch.zeros_like(index)
        slot[1:] = (index[1:] == index[:-1]).long()

        # Vectorized scatter operation
        children[index, slot] = inputs

        return children.reshape(num_nodes, 2 * self.in_dim)

    def update(self, aggr_out, x):
        """
        aggr_out: (num_nodes, 2*in_dim) concatenated children features
        x:        (num_nodes, in_dim)   parent features

        Returns:
            Fused parent representation → out_dim
        """
        fused = torch.cat([x, aggr_out], dim=-1)
        return fused

    def forward(self, x, edge_index):
        """
        x: (num_nodes, in_dim)
        edge_index: (2, num_edges), where edges point child -> parent

        Returns:
            Updated node embeddings (num_nodes, out_dim)
        """
        return self.propagate(edge_index, x=x)


class RecipientFinder(nn.Module):
    """
    A model that stacks several ParentChildFusionLayers followed by
    fully connected layers for prediction.

    Parameters
    ----------
    in_channels : int
        Dimensionality of the input node features.
    hidden_channels : int
        Dimensionality after the ParentChildFusionLayers.
    num_fusion_layers : int
        Number of ParentChildFusionLayers to apply sequentially.
    dropout : float
        Dropout probability for the fully connected classifier.
    """

    def __init__(self, in_channels, hidden_channels = 16,
                 num_fusion_layers=2, num_fc_layers = 2, dropout=0.3):

        super().__init__()

        # ----- Fusion Layers -----
        self.fusion_layers = nn.ModuleList()
        current_dim = in_channels

        for _ in range(num_fusion_layers):
            self.fusion_layers.append(ParentChildFusionLayer(in_dim=current_dim))
            current_dim = current_dim * 3 # 3 is coming from one parent and two children. If there are no children it is filled with zeros.

        # ----- Fully Connected Layers -----
        self.fc_layers = nn.ModuleList()
        self.dropout = dropout

        # First FC layer: (current_dim → hidden_channels) OR directly → 1
        if num_fc_layers == 1:
            self.fc_layers.append(nn.Linear(current_dim, 1))
        else:
            # First hidden layer
            self.fc_layers.append(nn.Linear(current_dim, hidden_channels))
            # Middle hidden FC layers
            for _ in range(num_fc_layers - 2):
                self.fc_layers.append(nn.Linear(hidden_channels, hidden_channels))
            # Final output layer
            self.fc_layers.append(nn.Linear(hidden_channels, 1))

    def forward(self, x, edge_index):
        """
        Apply stacked fusion layers followed by linear classifiers.
        """

        # ----- ParentChildFusionLayers -----
        for layer in self.fusion_layers:
            x = layer(x, edge_index)
            x = F.relu(x)

        # ----- Fully Connected Layers -----
        for i, layer in enumerate(self.fc_layers):
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = layer(x)
            if i < len(self.fc_layers) - 1:
                x = F.relu(x)

        return x.view(-1)


i = 0
model = RecipientFinder(in_channels = data[i].x.shape[1])
print(model)
model(data[i].x, data[i].edge_index)

RecipientFinder(
  (fusion_layers): ModuleList(
    (0-1): 2 x ParentChildFusionLayer()
  )
  (fc_layers): ModuleList(
    (0): Linear(in_features=36, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=1, bias=True)
  )
)


tensor([ 6.2573, -0.1835, -0.1707, -0.2310, -0.2280, -0.1361,  4.3763,  0.1117,
         4.6189], grad_fn=<ViewBackward0>)

In [3]:
from torch_geometric.loader import DataLoader

# === 1. Modell, Optimizer, Loss ===
model = RecipientFinder(in_channels=data[0].x.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Klassengewichte berechnen (gegen Ungleichgewicht)
all_labels = torch.cat([g.y for g in data])
ratio = (len(all_labels) - all_labels.sum()) / all_labels.sum()
pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

print(f"Pos Weight: {pos_weight.item():.2f}")

# === 2. Training & Evaluation ===

# Train/Test Split
random.shuffle(data)
split_idx = int(0.5 * len(data))
train_data = data[:split_idx]
test_data = data[split_idx:]

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
test_loader = DataLoader(train_data, batch_size=8)

def train():
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)
        loss = criterion(out, batch.y.float())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

@torch.no_grad()
def evaluate(loader):
    model.eval()
    total_correct = 0
    total_nodes = 0
    tp, fp, fn = 0, 0, 0

    for batch in loader:
        out = model(batch.x, batch.edge_index)
        preds = torch.sigmoid(out) > 0.5
        total_correct += (preds == batch.y.bool()).sum().item()
        total_nodes += batch.y.size(0)

        # Metriken für Klasse 1
        tp += ((preds == 1) & (batch.y == 1)).sum().item()
        fp += ((preds == 1) & (batch.y == 0)).sum().item()
        fn += ((preds == 0) & (batch.y == 1)).sum().item()

    acc = total_correct / total_nodes
    precision = tp / (tp + fp + 1e-8)
    recall = tp / (tp + fn + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)
    return acc, precision, recall, f1

# === 3. Training starten ===
for epoch in range(1, 21):
    loss = train()
    acc, prec, rec, f1 = evaluate(test_loader)
    print(f"Epoch {epoch:02d} | Loss: {loss:.4f} | Acc: {acc:.3f} | Prec: {prec:.3f} | Rec: {rec:.3f} | F1: {f1:.3f}")

Pos Weight: 5.53


/tmp/ipykernel_6782/535982600.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  pos_weight = torch.tensor((ratio**0.5), dtype=torch.float)


Epoch 01 | Loss: 0.9381 | Acc: 0.948 | Prec: 0.304 | Rec: 0.529 | F1: 0.386
Epoch 02 | Loss: 0.3411 | Acc: 0.951 | Prec: 0.316 | Rec: 0.511 | F1: 0.391
Epoch 03 | Loss: 0.2868 | Acc: 0.956 | Prec: 0.338 | Rec: 0.445 | F1: 0.384
Epoch 04 | Loss: 0.2643 | Acc: 0.958 | Prec: 0.343 | Rec: 0.398 | F1: 0.368
Epoch 05 | Loss: 0.2512 | Acc: 0.956 | Prec: 0.353 | Rec: 0.512 | F1: 0.418
Epoch 06 | Loss: 0.2412 | Acc: 0.964 | Prec: 0.400 | Rec: 0.365 | F1: 0.382
Epoch 07 | Loss: 0.2390 | Acc: 0.952 | Prec: 0.342 | Rec: 0.597 | F1: 0.435
Epoch 08 | Loss: 0.2379 | Acc: 0.955 | Prec: 0.350 | Rec: 0.534 | F1: 0.423
Epoch 09 | Loss: 0.2286 | Acc: 0.967 | Prec: 0.455 | Rec: 0.339 | F1: 0.389
Epoch 10 | Loss: 0.2286 | Acc: 0.952 | Prec: 0.336 | Rec: 0.565 | F1: 0.421
Epoch 11 | Loss: 0.2237 | Acc: 0.961 | Prec: 0.395 | Rec: 0.485 | F1: 0.435
Epoch 12 | Loss: 0.2263 | Acc: 0.950 | Prec: 0.332 | Rec: 0.630 | F1: 0.435
Epoch 13 | Loss: 0.2268 | Acc: 0.946 | Prec: 0.325 | Rec: 0.690 | F1: 0.442
Epoch 14 | L